**Imports**

In [11]:
# Imports
import pandas as pd
import random
import numpy as np

**Parameters**

In [2]:
MAX_TREE_DEPTH = 5
MIN_TREE_DEPTH = 2
POP_SIZE = 200
POP_GEN_METHOD = 'grow'

random.seed(10)

# Initialise generation method variables
grow = False
if POP_GEN_METHOD is 'grow':
    grow = True

<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\david\AppData\Local\Temp\ipykernel_15384\1671493786.py:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if POP_GEN_METHOD is 'grow':


**Dataset imports**

In [3]:
train_df = pd.read_csv(r'training.csv')
test_df = pd.read_csv(r'testing.csv')

In [4]:
# Printing the column headings to better understand the features
print(train_df.columns.values)

['Duration' 'Distance' 'Pickup_longitude' 'Pickup_latitude'
 'Dropoff_latitude' 'Dropoff_latitude.1' 'Haversine' 'Pmonth' 'Pickup_day'
 'Pickup_hour' 'Pickup_minute' 'Pickup_weekday' 'Dropoff_month'
 'Dropoff_day' 'Dropoff_hour' 'Dropoff_minute' 'Dropoff_weekday' 'Temp'
 'Precip' 'Wind' 'Humid' 'Solar' 'Snow' 'GroundTemp' 'Dust']


In [5]:
# Seperate values from data
# Remember: features = data used to reach output, while labels are the outputs themselves
train_labels = train_df[['Duration']]
test_labels = test_df[['Duration']]

train_features = train_df.drop('Duration',axis=1)
test_features = test_df.drop('Duration',axis=1)

print('Labels column: ' + str(train_labels.columns) + '\n\n')
print('Features columns: ' + str(train_features.columns))

Labels column: Index(['Duration'], dtype='object')


Features columns: Index(['Distance', 'Pickup_longitude', 'Pickup_latitude', 'Dropoff_latitude',
       'Dropoff_latitude.1', 'Haversine', 'Pmonth', 'Pickup_day',
       'Pickup_hour', 'Pickup_minute', 'Pickup_weekday', 'Dropoff_month',
       'Dropoff_day', 'Dropoff_hour', 'Dropoff_minute', 'Dropoff_weekday',
       'Temp', 'Precip', 'Wind', 'Humid', 'Solar', 'Snow', 'GroundTemp',
       'Dust'],
      dtype='object')


In [6]:
print(train_features.info())
print('Entries in "training features": ' + str(len(train_features)))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6720797 entries, 0 to 6720796
Data columns (total 24 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Distance            int64  
 1   Pickup_longitude    float64
 2   Pickup_latitude     float64
 3   Dropoff_latitude    float64
 4   Dropoff_latitude.1  float64
 5   Haversine           float64
 6   Pmonth              int64  
 7   Pickup_day          int64  
 8   Pickup_hour         int64  
 9   Pickup_minute       int64  
 10  Pickup_weekday      int64  
 11  Dropoff_month       int64  
 12  Dropoff_day         int64  
 13  Dropoff_hour        int64  
 14  Dropoff_minute      int64  
 15  Dropoff_weekday     int64  
 16  Temp                float64
 17  Precip              float64
 18  Wind                float64
 19  Humid               float64
 20  Solar               float64
 21  Snow                float64
 22  GroundTemp          float64
 23  Dust                float64
dtypes: float64(13), int64(11

In [7]:
print(train_labels.info())
print('Entries in "training labels": ' + str(len(train_labels)))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6720797 entries, 0 to 6720796
Data columns (total 1 columns):
 #   Column    Dtype
---  ------    -----
 0   Duration  int64
dtypes: int64(1)
memory usage: 51.3 MB
None
Entries in "training labels": 6720797


In [8]:
# Free up RAM
import gc

train_df = None
test_df = None

gc.collect()


0

**Terminal and Function set creation**

In [9]:
# Some basic functions for the function set
def add(x,y): return x + y
def sub(x,y): return x - y
def mul(x,y): return x * y
def div(x,y): return x / y

function_set = [add, sub, mul, div]

In [10]:
# Creation of terminal set
# Terminals are a range of constants, as made below
terminal_set = ['Duration', 'Distance', 'Pickup_longitude', 'Pickup_latitude',
    'Dropoff_latitude', 'Dropoff_latitude.1', 'Haversine', 'Pmonth', 'Pickup_day',
    'Pickup_hour', 'Pickup_minute', 'Pickup_weekday', 'Dropoff_month',
    'Dropoff_day', 'Dropoff_hour', 'Dropoff_minute', 'Dropoff_weekday', 'Temp',
    'Precip', 'Wind', 'Humid', 'Solar', 'Snow', 'GroundTemp', 'Dust']

**Tree definitions**

In [11]:
class GPTree:
    def __init__(self, data = None, left = None, right = None):
        self.data = data
        self.left = left
        self.right = right

    # Allows for either grow (if grow parameter is true) or full method (grow set to false)
    # Makes self into a new random tree
    def random_tree(self, grow, max_depth, depth = 0):
        if depth < max_depth or (depth < max_depth and not grow): 
            self.data = function_set[random.randint(0, len(function_set) - 1)]
        elif depth >= max_depth:   
            self.data = terminal_set[random.randint(0, len(terminal_set) - 1)]
        else:
            if random() > 0.5: # 50% chance
                self.data = terminal_set[random.randint(0, len(terminal_set) - 1)]
            else:
                self.data = function_set[random.randint(0, len(function_set) - 1)]
        if self.data in function_set:
            self.left = GPTree()
            self.left.random_tree(grow, max_depth, depth = depth + 1)            
            self.right = GPTree()
            self.right.random_tree(grow, max_depth, depth = depth + 1)

    # recursive depth-first evaluation, returns calculated value of tree - can be used on root or any subtree
    def evaluate_tree(self):
        if self.data in terminal_set:
            return self.data
        else:
            left_value = self.left.evaluate_tree()
            right_value = self.right.evaluate_tree()
            own_value = self.data(left_value, right_value)
            return own_value

    def print_tree(self):
        if self.data in terminal_set:
            print('TERMINAL: ' + str(self.data))
        elif self.data in function_set:
            print('FUNCTION: ' + str(self.data))
            print(self.left.print_tree())
            print(self.right.print_tree())
            pass

In [12]:
# Test for random_tree
test_tree = GPTree()
test_tree.random_tree(True, MAX_TREE_DEPTH)
print(test_tree.print_tree())

FUNCTION: <function add at 0x000001A1208D3700>
FUNCTION: <function div at 0x000001A120807310>
FUNCTION: <function div at 0x000001A120807310>
FUNCTION: <function add at 0x000001A1208D3700>
FUNCTION: <function sub at 0x000001A12079B160>
TERMINAL: Dropoff_hour
None
TERMINAL: Dropoff_minute
None
None
FUNCTION: <function mul at 0x000001A1208073A0>
TERMINAL: Humid
None
TERMINAL: Dropoff_latitude.1
None
None
None
FUNCTION: <function add at 0x000001A1208D3700>
FUNCTION: <function div at 0x000001A120807310>
TERMINAL: Pickup_minute
None
TERMINAL: Pickup_longitude
None
None
FUNCTION: <function sub at 0x000001A12079B160>
TERMINAL: GroundTemp
None
TERMINAL: Pickup_weekday
None
None
None
None
FUNCTION: <function add at 0x000001A1208D3700>
FUNCTION: <function div at 0x000001A120807310>
FUNCTION: <function sub at 0x000001A12079B160>
TERMINAL: Wind
None
TERMINAL: Pickup_weekday
None
None
FUNCTION: <function div at 0x000001A120807310>
TERMINAL: Dropoff_day
None
TERMINAL: Pickup_hour
None
None
None
FUNCT

**Initial population generation**

In [13]:
# Returns array containing initial tree population
# TODO - look for library that does the tree things but prettier
def init_population(): # Assuming not using ramped 50/50 - TODO, adjust later to allow this
    population = []
    for i in range(0, POP_SIZE):
        new_random_tree = GPTree()
        new_random_tree.random_tree(grow, MAX_TREE_DEPTH)
        population.append(new_random_tree)
    return population

In [14]:
# Test for init_population
test_pop = init_population()
print(len(test_pop))

200


**Fitness function**

In [ ]:
def raw_fitness_function(tree : GPTree, test_dataset_features, test_dataset_labels):
    tree_duration = tree.evaluate_tree()

    
    
    pass